In [66]:
# Importing libraries for interaction with AWS
import boto3
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr
# Importing library for request data from web
from requests import get
# Importing library for converting json data to csv data
import pandas as pd
# Importing library for manipulating with arrays
import numpy as np
# Importing library for visualizing data
import matplotlib.pyplot as plt
# Import other libraries
from decimal import Decimal
from pathlib import Path
import logging
from time import sleep
import os

## Construct class for manipulating with DynamoDB through AWS SDK for Python

In [53]:
class Currency:
    """"""
    def __init__(self, dyn_resource):
        """"""
        self.dyn_resource = dyn_resource
        self.table = None

    def create_table(self, table_name):
        """

        :param table_name:
        :return: The newly created table.
        """
        try:
            self.table = self.dyn_resource.create_table(
                TableName=table_name,
                KeySchema=[
                    {"AttributeName": "py_prt_key", "KeyType": "HASH"},  # Partition key
                    {"AttributeName": "py_srt_key", "KeyType": "RANGE"}  # Sort key
                ],
                AttributeDefinitions=[
                    {"AttributeName": "py_prt_key", "AttributeType": "S"},
                    {"AttributeName": "py_srt_key", "AttributeType": "N"}
                ],
                ProvisionedThroughput={"ReadCapacityUnits": 10, "WriteCapacityUnits": 10}
            )
            self.table.wait_until_exists()
        except ClientError as err:
            print("Couldn't create table {}. Here's why: {}: {}".format(table_name,
                  err.response["Error"]["Code"], err.response["Error"]["Message"]))
            raise
        else:
            return self.table

    def exist_table(self, table_name):
        """"""
        self.table = self.dyn_resource.Table(table_name)
        print("Use table {} which creation date time is: {}".format(self.table.name,
                                                                    self.table.creation_date_time))

    def add_item(self, py_prt_key, py_srt_key, currency, date, rate):
        """"""
        try:
            self.table.put_item(
                Item={
                    "py_prt_key": py_prt_key,
                    "py_srt_key": py_srt_key,
                    "currency": currency,
                    "date": date,
                    "rate": rate
                }
            )
        except ClientError as err:
            print("Couldn't add currency {} to table {}. Here's why: {}: {}".format(py_prt_key, self.table.name,
                  err.response["Error"]["Code"], err.response["Error"]["Message"]))
            raise
        else:
            print("Item {}-{} successfully added to the table {}".format(py_prt_key, py_srt_key,
                                                                       self.table.name))

    def multi_write_items(self, items):
        """"""
        try:
            with self.table.batch_writer() as writer:
                for i, item in enumerate(items):
                    writer.put_item(Item=item)
                    if i == 9:
                        # sleep(5)
                        break
        except ClientError as err:
            print("Couldn't load data into table {}. Here's why: {}: {}".format(self.table.name,
                  err.response["Error"]["Code"], err.response["Error"]["Message"]))
            raise
        else:
            print("All items successfully added to the table {}".format(self.table.name))

    def delete_item(self, py_prt_key, py_srt_key):
        """"""
        try:
            self.table.delete_item(Key={"py_prt_key": py_prt_key, "py_srt_key": py_srt_key})
        except ClientError as err:
            print("Couldn't delete currency {}. Here's why: {}: {}".format(py_prt_key,
                  err.response["Error"]["Code"], err.response["Error"]["Message"]))
            raise
        else:
            print("Item {}-{} successfully delete from the table {}".format(py_prt_key, py_srt_key,
                                                                            self.table.name))

    def query_table(self, py_prt_key, srt_gt_key):
        """"""
        try:
            response = self.table.query(
                KeyConditionExpression=(
                        Key("py_prt_key").eq(py_prt_key) &
                        Key("py_srt_key").gt(srt_gt_key)
                )
            )
        except ClientError as err:
            if err.response["Error"]["Code"] == "ValidationException":
                print("There's a validation error. Here's the message: {}: {}".format(
                      err.response["Error"]["Code"], err.response["Error"]["Message"]))
            else:
                print("Couldn't query for table {}. Here's why: {}: {}".format(self.table.name,
                      err.response["Error"]["Code"], err.response["Error"]["Message"]))
                raise
        else:
            return response["Items"]

## Creating table on Amazon DynamoDB

In [4]:
# Define boto3 DynamoDB resource
dynamodb = boto3.resource("dynamodb")

In [54]:
# Define class with all methods for manipulating with
aws_py_class = Currency(dynamodb)

In [7]:
# Creating table
aws_py_class.create_table("aws_py_table")

dynamodb.Table(name='aws_py_table')

## Adding items to the table

In [55]:
# Use existing table
aws_py_class.exist_table("aws_py_table")

Use table aws_py_table which creation date time is: 2023-04-22 06:57:25.661000+00:00


### Adding an item per query

In [7]:
# Adding one item to the table
aws_py_class.add_item("eur", 0, "Euro", [1, 1, 2021], Decimal(str(34.7396)))

Item eur-0 successfully put to the table aws_py_table


## Deleting an item from the table

In [39]:
# Adding another item to the table
aws_py_class.add_item("eur", 1, "Euro", [5, 1, 2021], Decimal(str(34.9389)))

Item eur-1 successfully put to the table aws_py_table


In [56]:
# Deleting one item from table
aws_py_class.delete_item("eur", 0)

Item eur-0 successfully delete from the table aws_py_table


### Multi-writing an items for the table

In [43]:
# Deleting last item from table
aws_py_class.delete_item("eur", 1)

Item eur-1 successfully delete from the table aws_py_table


In [57]:
# Read currency data from csv file
csv_path = "output/eur_currency.csv"
eur_df = pd.read_csv(csv_path)

In [58]:
# View size of DataFrame
eur_df.size

2555

In [59]:
# View on DataFrame
eur_df.head()

,Unnamed: 0,date,currency,rate,units,rate_per_unit,calcdate
0,0,01.01.2021,Euro,34.7396,1,34.7396,30.12.2020
1,1,02.01.2021,Euro,34.7396,1,34.7396,30.12.2020
2,2,03.01.2021,Euro,34.7396,1,34.7396,30.12.2020
3,3,04.01.2021,Euro,34.7396,1,34.7396,30.12.2020
4,4,05.01.2021,Euro,34.9389,1,34.9389,04.01.2021


In [60]:
# Drop unnecessary column
eur_df = eur_df.drop(columns=["Unnamed: 0", "units", "rate_per_unit", "calcdate"])

In [61]:
# View after drop column
eur_df.head()

,date,currency,rate
0,01.01.2021,Euro,34.7396
1,02.01.2021,Euro,34.7396
2,03.01.2021,Euro,34.7396
3,04.01.2021,Euro,34.7396
4,05.01.2021,Euro,34.9389


In [62]:
# Define items array with dict per item
items = np.array([])

In [63]:
# Fill items array with items
for i, row in enumerate(eur_df.values.tolist()):
    currency_item = {
        "py_prt_key": "eur",
        "py_srt_key": i,
        "currency": row[1],
        "date": [int(date) for date in row[0].split('.')],
        "rate": Decimal(str(row[2]))
    }
    items = np.concatenate([items, [currency_item]])

In [64]:
# View on one element of items
print(items[0])

{'py_prt_key': 'eur', 'py_srt_key': 0, 'currency': 'Euro', 'date': [1, 1, 2021], 'rate': Decimal('34.7396')}


In [65]:
# Multi-writing an items for the table through batch_writer method from boto3 for table
aws_py_class.multi_write_items(items)

All items successfully added to the table aws_py_table


## Searching item in the table

In [67]:
# Query all items with py_srt_key greater than 5
aws_py_class.query_table("eur", 5)

[{'date': [Decimal('7'), Decimal('1'), Decimal('2021')],
  'currency': 'Euro',
  'rate': Decimal('34.846'),
  'py_srt_key': Decimal('6'),
  'py_prt_key': 'eur'},
 {'date': [Decimal('8'), Decimal('1'), Decimal('2021')],
  'currency': 'Euro',
  'rate': Decimal('34.846'),
  'py_srt_key': Decimal('7'),
  'py_prt_key': 'eur'},
 {'date': [Decimal('9'), Decimal('1'), Decimal('2021')],
  'currency': 'Euro',
  'rate': Decimal('34.846'),
  'py_srt_key': Decimal('8'),
  'py_prt_key': 'eur'},
 {'date': [Decimal('10'), Decimal('1'), Decimal('2021')],
  'currency': 'Euro',
  'rate': Decimal('34.846'),
  'py_srt_key': Decimal('9'),
  'py_prt_key': 'eur'}]